Considère ce nouveau problème de satisfiabilité : 

De retour de la planète Pincus, tu souhaites planifier un voyage de cinq jours à Quanta, un pays gigantesque. Comme le temps est limité, tu souhaites optimiser ton voyage de façon à visiter le plus d'endroits possibles. Tu demandes alors au sage de Quanta, qui connait très bien le pays, de t'aider dans ta planification. Il te donne les affirmations suivantes : 

1. Si tu arrives jusqu'au désert, tu ne peux pas aller voir la forêt : **Désert → ¬Forêt**
2. L'océan est un paysage à ne pas manquer, tu dois absolument y aller : **Océan**
3. Si tu visites le lac, tu ne peux pas te rendre à la capitale : **Lac → ¬Capitale**
4. Si tu visites la capitale, tu ne peux pas aller voir l'océan : **Capitale → ¬Océan**
5. Si tu vas voir le lac, il faut absolument que tu passes par la forêt : c'est just à côté! **Lac → Forêt**

Utilises l'algorithme de Grover pour déterminer les solutions du problème, c'est à dire les combinaisons possibles d'endroits que tu vas visiter lors de ton voyage à Quanta. Quelle est la combinaison la plus grande d'endroits que tu iras voir? 

Indice : La formule logique du problème, sous forme normale conjonctive, est en annexe à la question, avec 
d = désert, c = capitale, l = lac, f = forêt,  o = océan. 

$$
f(d, c, l, f, o) = (\neg d \lor \neg f) \land o \land (\neg l \lor \neg c) \land (\neg c \lor \neg o) \land (\neg l \lor f)
$$

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import XGate, ZGate

import os, sys
sys.path.append(os.getcwd())

def variable_name_to_int(var_name):
    """This function returns an integer associated with a given variable

    Args:
        var_name (str): the current variable. For example : x1.

    Returns:
        int: an integer associated with the variable name. For example :0.
    """
    if var_name == "x0":
        return 0
    if var_name == "x1":
        return 1
    if var_name == "x2":
        return 2
    if var_name == "x3":
        return 3
    if var_name == "x4":
        return 4
    return -1


def get_variable_list_from_clause(disj_clause):
    """This function returns a list of integers associated with the variables in a given disjunction clause

    Args:
        disj_clause (dict): the current disjunction clause. For example : {'x0' : False, 'x3' : True, 'x1' : True}.

    Returns:
        list: a list of integers associated with the variables in disj_clause. For example : [0, 3, 1].
    """
    variables = []
    for var_name in disj_clause:
        variables.append(variable_name_to_int(var_name=var_name))
    return variables


def get_disjunction_qubits(disj_clause, clause_qubit, var_qubits):
    """This function returns the right qubits to which you want to apply the disjunction clause.

    For example, if we are looking at the first clause, we would have the following parameters :

        -   disj_clause = {'x2' : True, 'x0' : True, 'x3' : True}.
        -   clause_qubit = 0. The first clause is reprensented by the index 0 and the last one (seventh), by 6.
        -   var_qubits = qubits associated with variables x2, x0 and x3

        and we would return :

        -   disjunction_qubits = [x2, x0, x3, c4]. A list of qubits representing variables x2, x0 and x3 and clause 0.

    Args:
        circuit (QuantumCircuit): the oracle circuit from which we get the right qubits
        disj_clause (dict): the current disjunction clause
        ancillar_id (int): variable that represents the ID of current clause

    Returns:
        list: Returns a list of the qubits that are involved in the disjunction clause
    """
    variables = get_variable_list_from_clause(disj_clause)

    disjunction_qubits = []

    for var in variables:
        disjunction_qubits.append(var_qubits[var])

    disjunction_qubits.append(clause_qubit)

    return disjunction_qubits


def get_disjunction_control_state(disj_clause):
    """This function returns the control state of the multi controlled x-gate according to a disjunction clause.

    Args:
        disj_clause (dict): the current disjunction clause. For example : {'x1' : False, 'x4' : True, 'x2' : True}.

    Returns:
        str : Returns a string containing the control state. For example : '001'. (Little endian notation)
    """
    ctrl_state = ""
    # Get the control state according to given clause
    for var_name in disj_clause:
        state = disj_clause[var_name]
        if state:
            ctrl_state = "0" + ctrl_state
        else:
            ctrl_state = "1" + ctrl_state
    return ctrl_state


# Définition des clauses pour le problème de Quanta
# Variables:
# x0 = désert (d)
# x1 = capitale (c)
# x2 = lac (l)
# x3 = forêt (f)
# x4 = océan (o)

clauses = []

# 1- Si tu arrives jusqu'au désert, tu ne peux pas aller voir la forêt.
# Désert → ¬Forêt équivaut à ¬d ∨ ¬f
# (¬x0 ∨ ¬x3)
clauses.append({'x0': False, 'x3': False})

# 2- L'océan est un paysage à ne pas manquer, tu dois absolument y aller.
# o (qui équivaut à o ∨ o pour la forme CNF)
clauses.append({'x4': True})

# 3- Si tu visites le lac, tu ne peux pas te rendre à la capitale.
# Lac → ¬Capitale équivaut à ¬l ∨ ¬c
# (¬x2 ∨ ¬x1)
clauses.append({'x2': False, 'x1': False})

# 4- Si tu visites la capitale, tu ne peux pas aller voir l'océan.
# Capitale → ¬Océan équivaut à ¬c ∨ ¬o
# (¬x1 ∨ ¬x4)
clauses.append({'x1': False, 'x4': False})

# 5- Si tu vas voir le lac, il faut absolument que tu passes par la forêt.
# Lac → Forêt équivaut à ¬l ∨ f
# (¬x2 ∨ x3)
clauses.append({'x2': False, 'x3': True})

# x0 : désert, x1 : capitale, x2 : lac, x3 : forêt, x4 : océan
nb_variables = 5
# 5 clauses
nb_clauses = len(clauses)
print(f"Nombre de variables : {nb_variables}")
print(f"Nombre de clauses : {nb_clauses}")

# Transformer une disjonction en une porte
def logical_disjunction_to_gate(disj_clause):
    # Nombre de variables dans la clause
    nb_disj_variables = len(disj_clause)
    
    # Nombre de qubits dans la porte de disjonction (nb_variables + 1 qubit ancillaire)
    nb_qubits = nb_disj_variables + 1
    disj_qc = QuantumCircuit(nb_qubits)
    qubits = disj_qc.qubits
    
    # Obtenir le bon état de controle pour la porte multi-controle X
    ctrl_state = get_disjunction_control_state(disj_clause)
    
    # Créer une porte multi-controle X avec le bon nombre de qubits
    mc_xgate = XGate().control(num_ctrl_qubits=nb_disj_variables, ctrl_state = ctrl_state)
    disj_qc.append(mc_xgate, qubits)
    
    # Ajouter une porte X au qubit à la position -1 (dernière), le qubit ancillaire
    disj_qc.x(qubits[-1])
    
    # Transformer le circuit de disjonction en une porte avec un nom, mcx
    disj_gate = disj_qc.to_gate(label='mcx')
    return disj_gate


# Créer des registres quantiques pour les variables et les clauses
var_qubits = QuantumRegister(nb_variables, name='x')
clause_qubits = QuantumRegister(nb_clauses, name='c')

# Construire le circuit de clauses
clauses_circuit = QuantumCircuit(var_qubits, clause_qubits)

# Ajouter chaque clause de disjonction comme une porte en utilisant une boucle
for i in range(nb_clauses):
    # Convertir la disjonction en porte
    gate = logical_disjunction_to_gate(clauses[i])
    # Sélectionner les qubits associés à la clause i
    c_qubits = get_disjunction_qubits(clauses[i], clause_qubits[i], var_qubits)
    # Ajouter la porte au circuit de clauses
    clauses_circuit.append(gate, c_qubits)

# Afficher le circuit : 
# clauses_circuit.decompose(gates_to_decompose=['mcx'], reps=2).draw(output='mpl')


# Construire le circuit de l'oracle
oracle_circuit = QuantumCircuit(var_qubits, clause_qubits)

# Ajouter le circuit de clauses 
oracle_circuit.append(clauses_circuit.to_gate(label='clauses_circuit'), clauses_circuit.qubits)

# Ajouter la porte multi-controle Z
mc_z_gate = ZGate().control(nb_clauses - 1)
oracle_circuit.append(mc_z_gate, clause_qubits)

# Ajouter l'inverse du circuit de clauses
oracle_circuit.append(clauses_circuit.reverse_ops().to_gate(label='clauses_circuit'), oracle_circuit.qubits)

# Afficher le circuit
# oracle_circuit.decompose(gates_to_decompose=['clauses_circuit', 'mcx'], reps=2).draw(output='mpl')

# Construire le circuit de diffuseur
diffuser_circuit = QuantumCircuit(var_qubits)

# Ajouter des portes H et X pour chaque qubit du diffuseur
diffuser_circuit.h(range(nb_variables))
diffuser_circuit.x(range(nb_variables))

# Ajouter une multi-controle Z  
mc_z_gate = ZGate().control(nb_variables - 1)
diffuser_circuit.append(mc_z_gate, var_qubits)

# Ajouter des portes X et H pour chaque qubit du diffuseur
diffuser_circuit.x(range(nb_variables))
diffuser_circuit.h(range(nb_variables))

# Afficher le circuit
# diffuser_circuit.draw(output='mpl')

# Construire le circuit de Grover
c_bits = ClassicalRegister(nb_variables)
grover_circuit = QuantumCircuit(var_qubits, clause_qubits, c_bits)

# Ajouter des portes H pour chaque variable
grover_circuit.h(range(nb_variables))

# Identifier le nombre d'iterations
# Pour N = 2^5 = 32, le nombre optimal est environ π/4 * sqrt(32) ≈ 4.4 ≈ 4
nb_iterations = 3

# Ajouter autant d'oracles et de diffuseurs qu'il y a de nombre d'itérations
for it in range(nb_iterations):
    grover_circuit.append(oracle_circuit.to_gate(label='oracle'), grover_circuit.qubits)
    grover_circuit.barrier(grover_circuit.qubits)
    grover_circuit.append(diffuser_circuit.to_gate(label='diffusor'), grover_circuit.qubits[0:nb_variables])
    
# Ajouter les mesures pour l'evaluation du circuit
grover_circuit.measure(var_qubits, c_bits)

# Afficher le circuit
# grover_circuit.decompose(
#     gates_to_decompose=['oracle', 'clauses_circuit', 'diffusor', 'mcx'], 
#     reps=3).draw(output='mpl', scale=0.8)

from qiskit_aer import AerSimulator
from qiskit import transpile

# Prepaper une simulation pour rouler et mesurer la solution
def run_circuit(circ: QuantumCircuit) -> dict:
    """
    Run a quantum circuit on the AerSimulator and return the counts
    @param circ: QuantumCircuit to run
    @return: dictionary of measurement results and their counts
    """
    simulator = AerSimulator()
    circ = transpile(circ, simulator)
    result = simulator.run(circ, shots=1000).result()
    return result.get_counts(circ)

# Executer le circuit et obtenir le compte de solutions
counts = run_circuit(grover_circuit)

print("\nRésultats des mesures:")
print(counts)

# Afficher l'histogramme de comptes
plot_histogram(counts)

# Analyser les solutions
print("\n" + "="*60)
print("ANALYSE DES SOLUTIONS")
print("="*60)
print("\nRappel des variables (notation little endian):")
print("Position 0 (droite): x0 = désert")
print("Position 1: x1 = capitale")
print("Position 2: x2 = lac")
print("Position 3: x3 = forêt")
print("Position 4 (gauche): x4 = océan")
print("\nFormat: |x4 x3 x2 x1 x0>")

# Trier les résultats par nombre d'occurrences
sorted_counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)

print("\n" + "-"*60)
print("Top 5 des solutions les plus probables:")
print("-"*60)

for i, (state, count) in enumerate(sorted_counts[:5]):
    # Conversion: le bit le plus à droite est x0
    x0 = int(state[-1])  # désert
    x1 = int(state[-2])  # capitale
    x2 = int(state[-3])  # lac
    x3 = int(state[-4])  # forêt
    x4 = int(state[-5]) if len(state) >= 5 else 0  # océan
    
    destinations = []
    if x0: destinations.append("Désert")
    if x1: destinations.append("Capitale")
    if x2: destinations.append("Lac")
    if x3: destinations.append("Forêt")
    if x4: destinations.append("Océan")
    
    print(f"\n{i+1}. État '{state}' - {count} occurrences")
    print(f"   Variables: d={x0}, c={x1}, l={x2}, f={x3}, o={x4}")
    print(f"   Destinations visitées ({len(destinations)}): {', '.join(destinations) if destinations else 'Aucune'}")

# Trouver la combinaison avec le plus grand nombre de destinations
max_destinations = 0
best_states = []

for state, count in counts.items():
    x0 = int(state[-1])
    x1 = int(state[-2])
    x2 = int(state[-3])
    x3 = int(state[-4])
    x4 = int(state[-5]) if len(state) >= 5 else 0
    
    num_destinations = x0 + x1 + x2 + x3 + x4
    
    if num_destinations > max_destinations:
        max_destinations = num_destinations
        best_states = [(state, count)]
    elif num_destinations == max_destinations:
        best_states.append((state, count))

print("\n" + "="*60)
print(f"RÉPONSE: Nombre maximum de destinations = {max_destinations}")
print("="*60)

for state, count in best_states:
    x0 = int(state[-1])
    x1 = int(state[-2])
    x2 = int(state[-3])
    x3 = int(state[-4])
    x4 = int(state[-5]) if len(state) >= 5 else 0
    
    destinations = []
    if x0: destinations.append("Désert")
    if x1: destinations.append("Capitale")
    if x2: destinations.append("Lac")
    if x3: destinations.append("Forêt")
    if x4: destinations.append("Océan")
    
    print(f"\nÉtat '{state}' ({count} occurrences)")
    print(f"Destinations: {', '.join(destinations)}")

# Fonction pour vérifier si un état satisfait toutes les contraintes
def is_valid_solution(x0, x1, x2, x3, x4):
    """
    Vérifie si une configuration satisfait toutes les contraintes
    x0=désert, x1=capitale, x2=lac, x3=forêt, x4=océan
    """
    # Contrainte 1: ¬d ∨ ¬f (pas désert ET forêt)
    c1 = not (x0 == 1 and x3 == 1)
    
    # Contrainte 2: o (océan obligatoire)
    c2 = (x4 == 1)
    
    # Contrainte 3: ¬l ∨ ¬c (pas lac ET capitale)
    c3 = not (x2 == 1 and x1 == 1)
    
    # Contrainte 4: ¬c ∨ ¬o (pas capitale ET océan)
    c4 = not (x1 == 1 and x4 == 1)
    
    # Contrainte 5: ¬l ∨ f (si lac alors forêt)
    c5 = not (x2 == 1 and x3 == 0)
    
    return c1 and c2 and c3 and c4 and c5

# Analyser uniquement les solutions valides
print("\n" + "="*60)
print("SOLUTIONS VALIDES SEULEMENT")
print("="*60)

valid_solutions = []
for state, count in counts.items():
    x0 = int(state[-1])  # désert
    x1 = int(state[-2])  # capitale
    x2 = int(state[-3])  # lac
    x3 = int(state[-4])  # forêt
    x4 = int(state[-5]) if len(state) >= 5 else 0  # océan
    
    if is_valid_solution(x0, x1, x2, x3, x4):
        num_destinations = x0 + x1 + x2 + x3 + x4
        valid_solutions.append((state, count, num_destinations, x0, x1, x2, x3, x4))

# Trier par nombre de destinations puis par occurrences
valid_solutions.sort(key=lambda x: (x[2], x[1]), reverse=True)

print(f"\nNombre de solutions valides trouvées : {len(valid_solutions)}")

for i, (state, count, num_dest, x0, x1, x2, x3, x4) in enumerate(valid_solutions[:10]):
    destinations = []
    if x0: destinations.append("Désert")
    if x1: destinations.append("Capitale")
    if x2: destinations.append("Lac")
    if x3: destinations.append("Forêt")
    if x4: destinations.append("Océan")
    
    print(f"\n{i+1}. État '{state}' - {count} occurrences")
    print(f"   Variables: d={x0}, c={x1}, l={x2}, f={x3}, o={x4}")
    print(f"   Destinations ({num_dest}): {', '.join(destinations)}")

if valid_solutions:
    max_dest = valid_solutions[0][2]
    print(f"\n{'='*60}")
    print(f"RÉPONSE CORRECTE: Nombre maximum de destinations = {max_dest}")
    print(f"{'='*60}")

Nombre de variables : 5
Nombre de clauses : 5

Résultats des mesures:
{'01100': 16, '00000': 20, '10010': 23, '00010': 16, '01000': 18, '11011': 29, '01111': 19, '11110': 29, '11010': 29, '00001': 32, '00011': 18, '11000': 79, '10100': 20, '00111': 23, '01011': 26, '10101': 32, '00101': 31, '00100': 26, '11100': 71, '01010': 26, '01101': 21, '10111': 20, '01110': 23, '10011': 25, '00110': 33, '11101': 30, '11001': 29, '10000': 90, '10001': 62, '10110': 28, '11111': 33, '01001': 23}

ANALYSE DES SOLUTIONS

Rappel des variables (notation little endian):
Position 0 (droite): x0 = désert
Position 1: x1 = capitale
Position 2: x2 = lac
Position 3: x3 = forêt
Position 4 (gauche): x4 = océan

Format: |x4 x3 x2 x1 x0>

------------------------------------------------------------
Top 5 des solutions les plus probables:
------------------------------------------------------------

1. État '10000' - 90 occurrences
   Variables: d=0, c=0, l=0, f=0, o=1
   Destinations visitées (1): Océan

2. État '